### Ingest data on Google Earth Engine (WRI bucket)

* Purpose of script: This notebook will upload the geotiff files from the Google Cloud Storage to the WRI/aqueduct earthengine bucket. 
* Author: Rutger Hofste
* Kernel used: python36
* Date created: 20170802

## Preparation

* Authorize earthengine by running in your terminal: `earthengine authenticate`  
* you need to have access to the WRI-Aquaduct (yep a Google employee made a typo) bucket to ingest the data. Rutger can grant access to write to this folder. 
* Have access to the Google Cloud Storage Bucker

Run in your terminal `gcloud config set project aqueduct30`

In [1]:
import subprocess
import datetime
import os
import time
from datetime import timedelta
import re
import pandas as pd

## Settings

In [2]:
GCS_BASE = "gs://aqueduct30_v01/Y2017M08D02_RH_Upload_to_GoogleCS_V01/"

In [3]:
EE_BASE = "projects/WRI-Aquaduct/PCRGlobWB20V07"

## Functions

In [4]:
def splitKey(key):
    # will yield the root file code and extension of a set of keys
    prefix, extension = key.split(".")
    fileName = prefix.split("/")[-1]
    parameter = fileName[:-12]
    month = fileName[-2:] #can also do this with regular expressions if you like
    year = fileName[-7:-3]
    identifier = fileName[-11:-8]
    outDict = {"fileName":fileName,"extension":extension,"parameter":parameter,"month":month,"year":year,"identifier":identifier}
    return outDict

def splitParameter(parameter):
    #values = parameter.split("_")
    values = re.split("_|-", parameter) #soilmoisture uses a hyphen instead of underscore between the years
    keys = ["geographic_range","temporal_range","indicator","temporal_resolution","units","spatial_resolution","temporal_range_min","temporal_range_max"]
    # ['global', 'historical', 'PDomWN', 'month', 'millionm3', '5min', '1960', '2014']
    outDict = dict(zip(keys, values))
    outDict["parameter"] = parameter
    return outDict


## Script

In [5]:
command = ("earthengine create folder %s") %EE_BASE

In [6]:
print(command)

earthengine create folder projects/WRI-Aquaduct/PCRGlobWB20V07


In [7]:
subprocess.check_output(command,shell=True)

''

In [8]:
command = ("/opt/google-cloud-sdk/bin/gsutil ls %s") %(GCS_BASE)

In [9]:
keys = subprocess.check_output(command,shell=True)

In [10]:
keys = keys.decode('UTF-8').splitlines()

Removing first item from the list. The first item contains a folder without file name

In [11]:
keys2 = keys[1:]

In [12]:
df = pd.DataFrame()
i = 0
for key in keys2:
    i = i+1
    outDict = splitKey(key)
    df2 = pd.DataFrame(outDict,index=[i])
    df = df.append(df2)    

In [13]:
df.head()

,extension,fileName,identifier,month,parameter,year
1,tif,global_historical_PDomWN_month_millionm3_5min_...,000,01,global_historical_PDomWN_month_millionm3_5min_...,1960
2,tif,global_historical_PDomWN_month_millionm3_5min_...,001,02,global_historical_PDomWN_month_millionm3_5min_...,1960
3,tif,global_historical_PDomWN_month_millionm3_5min_...,002,03,global_historical_PDomWN_month_millionm3_5min_...,1960
4,tif,global_historical_PDomWN_month_millionm3_5min_...,003,04,global_historical_PDomWN_month_millionm3_5min_...,1960
5,tif,global_historical_PDomWN_month_millionm3_5min_...,004,05,global_historical_PDomWN_month_millionm3_5min_...,1960


In [14]:
df.tail()

,extension,fileName,identifier,month,parameter,year
9286,tif,global_historical_soilmoisture_month_meter_5mi...,679,08,global_historical_soilmoisture_month_meter_5mi...,2014
9287,tif,global_historical_soilmoisture_month_meter_5mi...,680,09,global_historical_soilmoisture_month_meter_5mi...,2014
9288,tif,global_historical_soilmoisture_month_meter_5mi...,681,10,global_historical_soilmoisture_month_meter_5mi...,2014
9289,tif,global_historical_soilmoisture_month_meter_5mi...,682,11,global_historical_soilmoisture_month_meter_5mi...,2014
9290,tif,global_historical_soilmoisture_month_meter_5mi...,683,12,global_historical_soilmoisture_month_meter_5mi...,2014


In [15]:
df.shape

(9290, 6)

In [16]:
parameters = df.parameter.unique()

In [17]:
print(parameters)

[u'global_historical_PDomWN_month_millionm3_5min_1960_2014'
 u'global_historical_PDomWN_year_millionm3_5min_1960_2014'
 u'global_historical_PDomWW_month_millionm3_5min_1960_2014'
 u'global_historical_PDomWW_year_millionm3_5min_1960_2014'
 u'global_historical_PIndWN_month_millionm3_5min_1960_2014'
 u'global_historical_PIndWN_year_millionm3_5min_1960_2014'
 u'global_historical_PIndWW_month_millionm3_5min_1960_2014'
 u'global_historical_PIndWW_year_millionm3_5min_1960_2014'
 u'global_historical_PIrrWN_month_millionm3_5min_1960_2014'
 u'global_historical_PIrrWN_year_millionm3_5min_1960_2014'
 u'global_historical_PIrrWW_month_millionm3_5min_1960_2014'
 u'global_historical_PIrrWW_year_millionm3_5min_1960_2014'
 u'global_historical_PLivWN_month_millionm3_5min_1960_2014'
 u'global_historical_PLivWN_year_millionm3_5min_1960_2014'
 u'global_historical_PLivWW_month_millionm3_5min_1960_2014'
 u'global_historical_PLivWW_year_millionm3_5min_1960_2014'
 u'global_historical_aqbasinwaterstress_month_di

We will store the geotiff images of each NetCDF4 file in imageCollections. The imageCollections will have the same name and content as the original NetCDF4files. 


In [18]:
for parameter in parameters:
    eeLocation = EE_BASE + "/" + parameter
    command = ("earthengine create collection %s") %eeLocation
    # Uncomment the following command if you run this script for the first time
    subprocess.check_output(command,shell=True)
    print(command)
    

earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PDomWN_month_millionm3_5min_1960_2014
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PDomWN_year_millionm3_5min_1960_2014
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PDomWW_month_millionm3_5min_1960_2014
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PDomWW_year_millionm3_5min_1960_2014
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PIndWN_month_millionm3_5min_1960_2014
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PIndWN_year_millionm3_5min_1960_2014
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PIndWW_month_millionm3_5min_1960_2014
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PIndWW_year_millionm3_5min_1960_2014
earthengine create c

Now that the folder and collections have been created we can start ingesting the data. It is crucial to store the relevant metadata with the images. 

In [19]:
df_parameter = pd.DataFrame()
i = 0
for parameter in parameters:
    i = i+1
    outDict_parameter = splitParameter(parameter)
    df_parameter2 = pd.DataFrame(outDict_parameter,index=[i])
    df_parameter = df_parameter.append(df_parameter2)   
    

In [20]:
df_parameter.head()

,geographic_range,indicator,parameter,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units
1,global,PDomWN,global_historical_PDomWN_month_millionm3_5min_...,5min,historical,2014,1960,month,millionm3
2,global,PDomWN,global_historical_PDomWN_year_millionm3_5min_1...,5min,historical,2014,1960,year,millionm3
3,global,PDomWW,global_historical_PDomWW_month_millionm3_5min_...,5min,historical,2014,1960,month,millionm3
4,global,PDomWW,global_historical_PDomWW_year_millionm3_5min_1...,5min,historical,2014,1960,year,millionm3
5,global,PIndWN,global_historical_PIndWN_month_millionm3_5min_...,5min,historical,2014,1960,month,millionm3


In [21]:
df_parameter.tail()

,geographic_range,indicator,parameter,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units
21,global,riverdischarge,global_historical_riverdischarge_month_m3secon...,5min,historical,2014,1960,month,m3second
22,global,riverdischarge,global_historical_riverdischarge_year_m3second...,5min,historical,2014,1960,year,m3second
23,global,runoff,global_historical_runoff_month_mmonth_5min_195...,5min,historical,2014,1958,month,mmonth
24,global,runoff,global_historical_runoff_year_myear_5min_1958_...,5min,historical,2014,1958,year,myear
25,global,soilmoisture,global_historical_soilmoisture_month_meter_5mi...,5min,historical,2014,1958,month,meter


In [22]:
df_parameter.shape

(25, 9)

In [23]:
df_complete = df.merge(df_parameter,how='left',left_on='parameter',right_on='parameter')

Adding NoData value, ingested_by and exportdescription

In [24]:
df_complete["nodata"] = -9999
df_complete["ingested_by"] ="RutgerHofste"
df_complete["exportdescription"] = df_complete["indicator"] + "_" + df_complete["temporal_resolution"]+"Y"+df_complete["year"]+"M"+df_complete["month"]

In [25]:
df_complete.head()

,extension,fileName,identifier,month,parameter,year,geographic_range,indicator,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units,nodata,ingested_by,exportdescription
0,tif,global_historical_PDomWN_month_millionm3_5min_...,000,01,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M01
1,tif,global_historical_PDomWN_month_millionm3_5min_...,001,02,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M02
2,tif,global_historical_PDomWN_month_millionm3_5min_...,002,03,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M03
3,tif,global_historical_PDomWN_month_millionm3_5min_...,003,04,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M04
4,tif,global_historical_PDomWN_month_millionm3_5min_...,004,05,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M05


In [26]:
df_complete.tail()

,extension,fileName,identifier,month,parameter,year,geographic_range,indicator,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units,nodata,ingested_by,exportdescription
9285,tif,global_historical_soilmoisture_month_meter_5mi...,679,08,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M08
9286,tif,global_historical_soilmoisture_month_meter_5mi...,680,09,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M09
9287,tif,global_historical_soilmoisture_month_meter_5mi...,681,10,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M10
9288,tif,global_historical_soilmoisture_month_meter_5mi...,682,11,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M11
9289,tif,global_historical_soilmoisture_month_meter_5mi...,683,12,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M12


In [27]:
list(df_complete.columns.values)

['extension',
 'fileName',
 'identifier',
 'month',
 'parameter',
 'year',
 'geographic_range',
 'indicator',
 'spatial_resolution',
 'temporal_range',
 'temporal_range_max',
 'temporal_range_min',
 'temporal_resolution',
 'units',
 'nodata',
 'ingested_by',
 'exportdescription']

In [30]:
def uploadEE(index,row):
    target = EE_BASE +"/"+ row.parameter + "/" + row.fileName
    source = GCS_BASE + row.fileName + "." + row.extension
    metadata = "--nodata_value=%s --time_start %s-%s-01 -p extension=%s -p filename=%s -p identifier=%s -p month=%s -p parameter=%s -p year=%s -p geographic_range=%s -p indicator=%s -p spatial_resolution=%s -p temporal_range=%s -p temporal_range_max=%s -p temporal_range_min=%s -p temporal_resolution=%s -p units=%s -p ingested_by=%s -p exportdescription=%s" %(row.nodata,row.year,row.month,row.extension,row.fileName,row.identifier,row.month,row.parameter,row.year,row.geographic_range,row.indicator,row.spatial_resolution,row.temporal_range,row.temporal_range_max,row.temporal_range_min, row.temporal_resolution, row.units, row.ingested_by, row.exportdescription)
    command = "/opt/anaconda3/bin/earthengine upload image --asset_id %s %s %s" % (target, source,metadata)
    try:
        response = subprocess.check_output(command, shell=True)
        outDict = {"command":command,"response":response,"error":0}
        df_errors2 = pd.DataFrame(outDict,index=[index])
        pass
    except:
        try:
            outDict = {"command":command,"response":response,"error":1}
        except:
            outDict = {"command":command,"response":-9999,"error":2}
        df_errors2 = pd.DataFrame(outDict,index=[index])
        print("error")
    return df_errors2



In [ ]:
df_errors = pd.DataFrame()
start_time = time.time()
for index, row in df_complete.iterrows():
    elapsed_time = time.time() - start_time 
    print(index,"%.2f" %((index/9289.0)*100), "elapsed: ", str(timedelta(seconds=elapsed_time)))
    df_errors2 = uploadEE(index,row)
    df_errors = df_errors.append(df_errors2)
    
    

(0, '0.00', 'elapsed: ', '0:00:00.003355')
(1, '0.01', 'elapsed: ', '0:00:01.238936')
(2, '0.02', 'elapsed: ', '0:00:02.544729')
(3, '0.03', 'elapsed: ', '0:00:03.745147')
(4, '0.04', 'elapsed: ', '0:00:05.353619')
(5, '0.05', 'elapsed: ', '0:00:06.558753')
(6, '0.06', 'elapsed: ', '0:00:07.781523')
(7, '0.08', 'elapsed: ', '0:00:08.937642')
(8, '0.09', 'elapsed: ', '0:00:10.165675')
(9, '0.10', 'elapsed: ', '0:00:11.411167')
(10, '0.11', 'elapsed: ', '0:00:12.607210')
(11, '0.12', 'elapsed: ', '0:00:14.144701')
(12, '0.13', 'elapsed: ', '0:00:15.277748')
(13, '0.14', 'elapsed: ', '0:00:16.626656')
(14, '0.15', 'elapsed: ', '0:00:17.800297')
(15, '0.16', 'elapsed: ', '0:00:19.018698')
(16, '0.17', 'elapsed: ', '0:00:20.262236')
(17, '0.18', 'elapsed: ', '0:00:22.021277')
(18, '0.19', 'elapsed: ', '0:00:23.437843')
(19, '0.20', 'elapsed: ', '0:00:24.596882')
(20, '0.22', 'elapsed: ', '0:00:25.719123')
(21, '0.23', 'elapsed: ', '0:00:26.904799')
(22, '0.24', 'elapsed: ', '0:00:29.361888'

(185, '1.99', 'elapsed: ', '0:03:53.738601')
(186, '2.00', 'elapsed: ', '0:03:54.865843')
(187, '2.01', 'elapsed: ', '0:03:56.016898')
(188, '2.02', 'elapsed: ', '0:03:57.077576')
(189, '2.03', 'elapsed: ', '0:03:58.206870')
(190, '2.05', 'elapsed: ', '0:03:59.446578')
(191, '2.06', 'elapsed: ', '0:04:00.665957')
(192, '2.07', 'elapsed: ', '0:04:01.813539')
(193, '2.08', 'elapsed: ', '0:04:02.969135')
(194, '2.09', 'elapsed: ', '0:04:04.175484')
(195, '2.10', 'elapsed: ', '0:04:05.462956')
(196, '2.11', 'elapsed: ', '0:04:06.687618')
(197, '2.12', 'elapsed: ', '0:04:08.107477')
(198, '2.13', 'elapsed: ', '0:04:09.314143')
(199, '2.14', 'elapsed: ', '0:04:10.607933')
(200, '2.15', 'elapsed: ', '0:04:11.832114')
(201, '2.16', 'elapsed: ', '0:04:13.001915')
(202, '2.17', 'elapsed: ', '0:04:14.168158')
(203, '2.19', 'elapsed: ', '0:04:15.450827')
(204, '2.20', 'elapsed: ', '0:04:16.601958')
(205, '2.21', 'elapsed: ', '0:04:17.766131')
(206, '2.22', 'elapsed: ', '0:04:18.957875')
(207, '2.2

(368, '3.96', 'elapsed: ', '0:07:45.768708')
(369, '3.97', 'elapsed: ', '0:07:46.950619')
(370, '3.98', 'elapsed: ', '0:07:48.082409')
(371, '3.99', 'elapsed: ', '0:07:49.245863')
(372, '4.00', 'elapsed: ', '0:07:50.386877')
(373, '4.02', 'elapsed: ', '0:07:51.675793')
(374, '4.03', 'elapsed: ', '0:07:52.830632')
(375, '4.04', 'elapsed: ', '0:07:54.046930')
(376, '4.05', 'elapsed: ', '0:07:55.208694')
(377, '4.06', 'elapsed: ', '0:07:56.434225')
(378, '4.07', 'elapsed: ', '0:07:57.525905')
(379, '4.08', 'elapsed: ', '0:07:58.780934')
(380, '4.09', 'elapsed: ', '0:07:59.965778')
(381, '4.10', 'elapsed: ', '0:08:01.247519')
(382, '4.11', 'elapsed: ', '0:08:02.447360')
(383, '4.12', 'elapsed: ', '0:08:03.729407')
(384, '4.13', 'elapsed: ', '0:08:05.054179')
(385, '4.14', 'elapsed: ', '0:08:06.210401')
(386, '4.16', 'elapsed: ', '0:08:07.391648')
(387, '4.17', 'elapsed: ', '0:08:08.549255')
(388, '4.18', 'elapsed: ', '0:08:09.702472')
(389, '4.19', 'elapsed: ', '0:08:10.848660')
(390, '4.2

(551, '5.93', 'elapsed: ', '0:11:32.734921')
(552, '5.94', 'elapsed: ', '0:11:33.957798')
(553, '5.95', 'elapsed: ', '0:11:35.176644')
(554, '5.96', 'elapsed: ', '0:11:36.349355')
(555, '5.97', 'elapsed: ', '0:11:37.471344')
(556, '5.99', 'elapsed: ', '0:11:38.646037')
(557, '6.00', 'elapsed: ', '0:11:39.770634')
(558, '6.01', 'elapsed: ', '0:11:40.907500')
(559, '6.02', 'elapsed: ', '0:11:42.064400')
(560, '6.03', 'elapsed: ', '0:11:43.225808')
(561, '6.04', 'elapsed: ', '0:11:44.402171')
(562, '6.05', 'elapsed: ', '0:11:45.574003')
(563, '6.06', 'elapsed: ', '0:11:46.826823')
(564, '6.07', 'elapsed: ', '0:11:48.095144')
(565, '6.08', 'elapsed: ', '0:11:49.253793')
(566, '6.09', 'elapsed: ', '0:11:50.316206')
(567, '6.10', 'elapsed: ', '0:11:51.494849')
(568, '6.11', 'elapsed: ', '0:11:53.470254')
(569, '6.13', 'elapsed: ', '0:11:54.612259')
(570, '6.14', 'elapsed: ', '0:11:56.002321')
(571, '6.15', 'elapsed: ', '0:11:57.214718')
(572, '6.16', 'elapsed: ', '0:11:58.365864')
(573, '6.1

(734, '7.90', 'elapsed: ', '0:15:30.735649')
(735, '7.91', 'elapsed: ', '0:15:32.011168')
(736, '7.92', 'elapsed: ', '0:15:33.254491')
(737, '7.93', 'elapsed: ', '0:15:34.401040')
(738, '7.94', 'elapsed: ', '0:15:35.465675')
(739, '7.96', 'elapsed: ', '0:15:36.620352')
(740, '7.97', 'elapsed: ', '0:15:37.830881')
(741, '7.98', 'elapsed: ', '0:15:39.027796')
(742, '7.99', 'elapsed: ', '0:15:40.170130')
(743, '8.00', 'elapsed: ', '0:15:44.790268')
(744, '8.01', 'elapsed: ', '0:15:45.960234')
(745, '8.02', 'elapsed: ', '0:15:47.296697')
(746, '8.03', 'elapsed: ', '0:15:48.859608')
(747, '8.04', 'elapsed: ', '0:15:50.021596')
(748, '8.05', 'elapsed: ', '0:15:51.338348')
(749, '8.06', 'elapsed: ', '0:15:52.610734')
(750, '8.07', 'elapsed: ', '0:15:55.630869')
(751, '8.08', 'elapsed: ', '0:15:56.827275')
(752, '8.10', 'elapsed: ', '0:15:57.913150')
(753, '8.11', 'elapsed: ', '0:15:59.069543')
(754, '8.12', 'elapsed: ', '0:16:00.844115')
(755, '8.13', 'elapsed: ', '0:16:02.125508')
(756, '8.1

...

(7158, '77.06', 'elapsed: ', '2:30:15.690691')
(7159, '77.07', 'elapsed: ', '2:30:16.931390')
(7160, '77.08', 'elapsed: ', '2:30:18.142564')
(7161, '77.09', 'elapsed: ', '2:30:19.442211')
(7162, '77.10', 'elapsed: ', '2:30:20.644426')
(7163, '77.11', 'elapsed: ', '2:30:21.806625')
(7164, '77.12', 'elapsed: ', '2:30:22.997980')
(7165, '77.13', 'elapsed: ', '2:30:24.164484')
(7166, '77.15', 'elapsed: ', '2:30:25.310064')
(7167, '77.16', 'elapsed: ', '2:30:26.466513')
(7168, '77.17', 'elapsed: ', '2:30:27.756163')
(7169, '77.18', 'elapsed: ', '2:30:29.467552')
(7170, '77.19', 'elapsed: ', '2:30:30.597613')
(7171, '77.20', 'elapsed: ', '2:30:31.749341')
(7172, '77.21', 'elapsed: ', '2:30:32.947265')
(7173, '77.22', 'elapsed: ', '2:30:34.076897')
(7174, '77.23', 'elapsed: ', '2:30:35.276881')
(7175, '77.24', 'elapsed: ', '2:30:36.475145')
(7176, '77.25', 'elapsed: ', '2:30:37.721320')
(7177, '77.26', 'elapsed: ', '2:30:38.959476')
(7178, '77.27', 'elapsed: ', '2:30:40.185643')
(7179, '77.28

(7333, '78.94', 'elapsed: ', '2:33:51.778691')
(7334, '78.95', 'elapsed: ', '2:33:53.034564')
(7335, '78.96', 'elapsed: ', '2:33:54.236373')
(7336, '78.98', 'elapsed: ', '2:33:55.423460')
(7337, '78.99', 'elapsed: ', '2:33:56.632509')
(7338, '79.00', 'elapsed: ', '2:33:57.787729')
(7339, '79.01', 'elapsed: ', '2:33:59.280566')
(7340, '79.02', 'elapsed: ', '2:34:00.444832')
(7341, '79.03', 'elapsed: ', '2:34:01.638951')
(7342, '79.04', 'elapsed: ', '2:34:02.913564')
(7343, '79.05', 'elapsed: ', '2:34:04.073427')
(7344, '79.06', 'elapsed: ', '2:34:05.285896')
(7345, '79.07', 'elapsed: ', '2:34:06.505458')
(7346, '79.08', 'elapsed: ', '2:34:07.655466')
(7347, '79.09', 'elapsed: ', '2:34:08.763522')
(7348, '79.10', 'elapsed: ', '2:34:09.953765')
(7349, '79.12', 'elapsed: ', '2:34:11.241383')
(7350, '79.13', 'elapsed: ', '2:34:12.423370')
(7351, '79.14', 'elapsed: ', '2:34:13.592628')
(7352, '79.15', 'elapsed: ', '2:34:14.828460')
(7353, '79.16', 'elapsed: ', '2:34:16.074501')
(7354, '79.17

(7508, '80.83', 'elapsed: ', '2:37:22.927327')
(7509, '80.84', 'elapsed: ', '2:37:24.166235')
(7510, '80.85', 'elapsed: ', '2:37:25.332393')
(7511, '80.86', 'elapsed: ', '2:37:26.552100')
(7512, '80.87', 'elapsed: ', '2:37:27.744043')
(7513, '80.88', 'elapsed: ', '2:37:29.028792')
(7514, '80.89', 'elapsed: ', '2:37:30.148083')
(7515, '80.90', 'elapsed: ', '2:37:31.324885')
(7516, '80.91', 'elapsed: ', '2:37:32.509133')
(7517, '80.92', 'elapsed: ', '2:37:33.640277')
(7518, '80.93', 'elapsed: ', '2:37:34.825759')
(7519, '80.95', 'elapsed: ', '2:37:36.222963')
(7520, '80.96', 'elapsed: ', '2:37:37.426333')
(7521, '80.97', 'elapsed: ', '2:37:38.823188')
(7522, '80.98', 'elapsed: ', '2:37:39.984827')
(7523, '80.99', 'elapsed: ', '2:37:41.194276')
(7524, '81.00', 'elapsed: ', '2:37:42.389732')
(7525, '81.01', 'elapsed: ', '2:37:43.601327')
(7526, '81.02', 'elapsed: ', '2:37:44.889953')
(7527, '81.03', 'elapsed: ', '2:37:46.094764')
(7528, '81.04', 'elapsed: ', '2:37:47.245693')
(7529, '81.05

(7683, '82.71', 'elapsed: ', '2:40:54.818835')
(7684, '82.72', 'elapsed: ', '2:40:55.920385')
(7685, '82.73', 'elapsed: ', '2:40:57.061944')
(7686, '82.74', 'elapsed: ', '2:40:58.280320')
(7687, '82.75', 'elapsed: ', '2:40:59.420892')
(7688, '82.76', 'elapsed: ', '2:41:00.573347')
(7689, '82.78', 'elapsed: ', '2:41:01.774359')
(7690, '82.79', 'elapsed: ', '2:41:02.949658')
(7691, '82.80', 'elapsed: ', '2:41:04.010208')
(7692, '82.81', 'elapsed: ', '2:41:05.163216')
(7693, '82.82', 'elapsed: ', '2:41:06.354107')
(7694, '82.83', 'elapsed: ', '2:41:07.618041')
(7695, '82.84', 'elapsed: ', '2:41:08.825484')
(7696, '82.85', 'elapsed: ', '2:41:10.020307')
(7697, '82.86', 'elapsed: ', '2:41:11.487601')
(7698, '82.87', 'elapsed: ', '2:41:12.646320')
(7699, '82.88', 'elapsed: ', '2:41:13.794167')
(7700, '82.89', 'elapsed: ', '2:41:15.003417')
(7701, '82.90', 'elapsed: ', '2:41:16.267497')
(7702, '82.92', 'elapsed: ', '2:41:17.443211')
(7703, '82.93', 'elapsed: ', '2:41:18.469633')
(7704, '82.94

(7858, '84.59', 'elapsed: ', '2:44:31.859681')
(7859, '84.61', 'elapsed: ', '2:44:33.034479')
(7860, '84.62', 'elapsed: ', '2:44:34.218791')
(7861, '84.63', 'elapsed: ', '2:44:35.412761')
(7862, '84.64', 'elapsed: ', '2:44:36.602330')
(7863, '84.65', 'elapsed: ', '2:44:37.836765')
(7864, '84.66', 'elapsed: ', '2:44:39.033464')
(7865, '84.67', 'elapsed: ', '2:44:40.213145')
(7866, '84.68', 'elapsed: ', '2:44:41.361218')
(7867, '84.69', 'elapsed: ', '2:44:42.573944')
(7868, '84.70', 'elapsed: ', '2:44:43.716294')
(7869, '84.71', 'elapsed: ', '2:44:44.926059')
(7870, '84.72', 'elapsed: ', '2:44:46.087082')
(7871, '84.73', 'elapsed: ', '2:44:47.264751')
(7872, '84.75', 'elapsed: ', '2:44:48.371158')
(7873, '84.76', 'elapsed: ', '2:44:49.522075')
(7874, '84.77', 'elapsed: ', '2:44:50.720786')
(7875, '84.78', 'elapsed: ', '2:44:51.966018')
(7876, '84.79', 'elapsed: ', '2:44:53.254262')
(7877, '84.80', 'elapsed: ', '2:44:54.447046')
(7878, '84.81', 'elapsed: ', '2:44:55.655220')
(7879, '84.82

In [37]:
!mkdir /volumes/data/temp

mkdir: cannot create directory '/volumes/data/temp': File exists


In [38]:
df_errors.to_csv("/volumes/data/temp/df_errors.csv")

In [39]:
!aws s3 cp  /volumes/data/temp/df_errors.csv s3://wri-projects/Aqueduct30/temp/df_errors.csv

upload: ../../../../data/temp/df_errors.csv to s3://wri-projects/Aqueduct30/temp/df_errors.csv


Retry the ones with errors

In [40]:
df_retry = df_errors.loc[df_errors['error'] != 0]

In [41]:
for index, row in df_retry.iterrows():
    response = subprocess.check_output(row.command, shell=True)
    

In [53]:
uniques = df_errors["error"].unique()